In [1]:
import VolumeAnalizer
import pandas as pd
reader=VolumeAnalizer.initialize()



In [2]:
companies_default=['TWTR','ATVI', 'ADBE', 'AMD', 'ALGN', 'ALXN', 'AMZN', 'AMGN', 'AAL', 'ADI', 'AAPL', 'AMAT', 'ASML', 'ADSK', 'ADP',
 'AVGO', 'BIDU', 'BIIB', 'BMRN', 'CDNS', 'CELG', 'CERN', 'CHKP', 'CHTR', 'CTRP', 'CTAS', 'CSCO', 'CTXS', 'CMCSA', 'COST', 
 'CSX', 'CTSH', 'DLTR', 'EA', 'EBAY', 'EXPE', 'FAST', 'FB', 'FISV', 'GILD', 'GOOG', 'GOOGL', 'HAS', 'HSIC', 'ILMN', 'INCY', 
 'INTC', 'INTU', 'ISRG', 'IDXX', 'JBHT', 'JD', 'KLAC', 'KHC', 'LRCX', 'LBTYA', 'LBTYK', 'LULU', 'MELI', 'MAR', 'MCHP', 'MDLZ',
  'MNST', 'MSFT', 'MU', 'MXIM', 'MYL', 'NTAP', 'NFLX', 'NTES', 'NVDA', 'NXPI', 'ORLY', 'PAYX', 'PCAR', 'BKNG', 'PYPL', 'PEP',
   'QCOM', 'REGN', 'ROST', 'SIRI', 'SWKS', 'SBUX', 'SNPS', 'TTWO', 'TSLA', 'TXN', 'TMUS', 'ULTA', 'UAL', 'VRSN', 'VRSK',
    'VRTX', 'WBA', 'WDC', 'WDAY', 'WYNN', 'XEL', 'XLNX']
companies1=['NVDA', 'AMD', 'TXN', 'AAPL', 'EA', 'ADSK','ADI','ALGN','CERN','CSCO']
describe,volume_data,dict_data=reader.getData(companies=companies_default,volume_data=True,all_data=True)


#Correlacion precios
df_correlation_by_moments={moment:pd.concat([data[moment].rename(company) for company,data in dict_data.items()],axis=1).pct_change(1).corr() for moment in ['open','high','low','close']}
print(df_correlation_by_moments['close'])
print(describe)



          TWTR      ATVI      ADBE       AMD      ALGN      ALXN      AMZN  \
TWTR  1.000000  0.268671  0.347037  0.404746  0.310700  0.219502  0.417988   
ATVI  0.268671  1.000000  0.301371  0.212855  0.125412  0.171314  0.266612   
ADBE  0.347037  0.301371  1.000000  0.339491  0.266551  0.166260  0.384349   
AMD   0.404746  0.212855  0.339491  1.000000  0.290286  0.206598  0.363484   
ALGN  0.310700  0.125412  0.266551  0.290286  1.000000  0.156427  0.279963   
...        ...       ...       ...       ...       ...       ...       ...   
WDC   0.351920  0.218114  0.207768  0.346249  0.220136  0.214909  0.284598   
WDAY  0.142955  0.181655  0.402936  0.121786  0.127922  0.082060  0.187238   
WYNN  0.347863  0.201471  0.212657  0.282907  0.266213  0.211037  0.346471   
XEL  -0.081955 -0.003290  0.032305 -0.131972 -0.081251 -0.061495 -0.067249   
XLNX  0.373030  0.238548  0.270139  0.435840  0.235177  0.274719  0.362383   

          AMGN       AAL       ADI  ...       UAL      VRSN    

In [2]:
max_mean_volumne=10000
condition_max_volume_mean=describe['mean']<max_mean_volumne


NameError: name 'describe' is not defined

In [3]:
from datetime import datetime
gt1=volume_data.index.to_series().diff()>pd.Timedelta(minutes=1)
data_diff=volume_data.groupby(gt1.cumsum()).diff()

v_notna=volume_data.notna()
valid_diff=v_notna&v_notna.shift(fill_value=False)

data_diff_valid=data_diff.where(valid_diff)
max_valid=data_diff_valid.abs().max()

#diff maxima permitida
max_diff_valid=1000000
import matplotlib.pyplot as plt




ax=max_valid.plot(kind='bar',figsize=(20,12),fontsize=14)
#plt.xticks(rotation=-45)
#grid on
plt.grid()
# set y=0
ax.axhline(max_diff_valid, color='red', lw=3)
ax.set_ylim(0,20000000)
ax.set_title('Cambios máximos por minuto en el Volumen',fontsize=20)
data_now=datetime.now().strftime(format='%Y-%m-%d')
ax.get_figure().savefig(f'images/Cambios Volumen 11-11-2019 to {data_now}')

NameError: name 'volume_data' is not defined

In [5]:
condition_max_diff=max_valid<max_diff_valid
condition_volume=condition_max_diff&condition_max_volume_mean


In [6]:
#Getting condition by correlation


In [23]:
valid_corr=df_correlation_by_moments['close'].loc[condition_volume,condition_volume]
valid_corr=valid_corr.assign(Sum_Correlation=valid_corr.sum(axis=1)).sort_values('Sum_Correlation',ascending = False)
m=valid_corr.ge(0.45).sum()>4
valid_corr.loc[m,m]

,AVGO,LRCX,MCHP,MXIM,SWKS,Sum_Correlation
MCHP,0.545999,0.503000,1.000000,0.639925,0.547207,16.472174
MXIM,0.479053,0.469246,0.639925,1.000000,0.467073,15.793487
AVGO,1.000000,0.417425,0.545999,0.479053,0.488295,15.479305
SWKS,0.488295,0.396579,0.547207,0.467073,1.000000,14.348445
LRCX,0.417425,1.000000,0.503000,0.469246,0.396579,13.903657


In [24]:
#AVGO,MXIM,MCHP ,POTENCIAL SWKS

In [1]:
df_correlation_by_moments['close']

NameError: name 'df_correlation_by_moments' is not defined